In [3]:
import numpy as np
from itertools import product
import math
import random
import copy

According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

#### ISING MODEL

In [7]:
class IsingModel:
    def __init__(self, lr=0.01):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        self.gaussian_scale = 0.01
        
    
    def fit_activations(self, activations_list):
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(activations_list)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # Calculate the mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
        # Calculate the correlation
        self.data_correlation = self.calculate_corr(self.data_activations_matrix)
        
        #Create random H and J matrices
        self.create_H_J()
        
         
    def fit_m_c(self, m, C):
        self.data_mean = m
        self.data_correlation = C
    
    def create_H_J(self, gaussian_scale = None):
        if gaussian_scale is None:
            gaussian_scale = self.gaussian_scale
            
        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        np.random.seed(3)
        self.J = np.random.normal(loc=0.0, scale=gaussian_scale, size=(self.num_spins, self.num_spins))
        self.lind = np.tril_indices_from(self.J)
        self.J[self.lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=gaussian_scale, size=self.num_spins)
    
    def set_H_J(self):
        self.H = H
        self.J = J        
        
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.0001
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Compute the statistics
            model_mean, model_correlation = self.get_statistics(prob_dict)


            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            # Take the step
            oldJ = copy.deepcopy(self.J)
            self.J = self.J + stepJ
            self.J[self.lind]=0

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Calculate the variation of J for the termination condition
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
            
            epoch += 1
            
        print('Final stats. TotalParamVariation', round(totalParamVariation, 8))
        print('Total epochs', epoch-1)
        print(model_correlation, 'C Model')
        print(self.data_correlation, 'C Data')

        print(model_mean, 'Average Model')
        print(self.data_mean, 'Average Data')
        print()

        print(self.H, 'H')
        print(self.J, 'J')

    # Calculate the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            for j in range(i+1, self.num_spins):
                energy += sigma[i]*sigma[j]*self.J[i][j]

        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    def get_delta_energy(self, i):
        #delta_energy = 2*(self.H[spin_to_flip]*self.sigma[spin_to_flip])
        
        delta_energy= 2 * (self.H[i]*self.sigma[i] \
                           + self.sigma[i] * np.dot(self.J[i, :], self.sigma) \
                           + self.sigma[i] * np.dot(self.J[:, i], self.sigma))
        
        return delta_energy

    # Calculates the correlation of a list of activations
    def calculate_corr(self, activations_matrix):
        num_samples_data = activations_matrix.shape[0]
        
        corr_matrix = np.zeros((self.num_spins, self.num_spins))
        for sigma in activations_matrix:
            # Calculate \sigma_i * \sigma_j
            for i in range(0, self.num_spins):
                for j in range(i+1, self.num_spins):
                    corr_matrix[i][j] += sigma[i]*sigma[j]
        corr_matrix = corr_matrix / float(num_samples_data)
        
        return corr_matrix
    
    def get_statistics(self, states_dict):
        
        # Calculate the partition function
        Z = sum(states_dict.values())
        
        # Returns the magnetization and correlation from a dictionary of activations {sigma, prob}
        magnetization =  np.zeros(self.num_spins)
        corr_matrix = np.zeros((self.num_spins,self.num_spins))
        # Calculate the expectation
        for sigma, count in states_dict.items():
            magnetization += np.dot(sigma,count/Z)
            for i in range(0, self.num_spins):
                for j in range(i+1, self.num_spins):
                    corr_matrix[i][j] += sigma[i]*sigma[j]*count/Z
        return magnetization, corr_matrix
    
    def metropolis_step(self):
        # Randomly select a spin (now it can only select 0)
        spin_to_flip = np.random.randint(self.num_spins)
        delta_energy = self.get_delta_energy(spin_to_flip)

        p = math.exp(-delta_energy)
        r = random.random()

        #print(delta_energy, r, p)
        if delta_energy <= 0 or r<=p:
            self.sigma[spin_to_flip] = -self.sigma[spin_to_flip]

    def metropolis_simulation(self, max_steps, initial_burn_in_len=0, inter_burn_in=0):
        # Generate a random initial spin configuration
        self.sigma = random.choices(self.spin_values, k=self.num_spins)        
        
        # Create a dictionary to count the frequencies
        samples_dict = {}
        for i in range(0, max_steps):
            # Take a metropolis step
            self.metropolis_step()
            
            # Save the state if the burn-in phase has passed
            if i >= initial_burn_in_len and (i%inter_burn_in)==0:
                if not tuple(copy.deepcopy(self.sigma)) in samples_dict: # If the state is new, create an entry
                    samples_dict[tuple(copy.deepcopy(self.sigma))] = 1
                else: # If the state already appeared, add one
                    samples_dict[tuple(copy.deepcopy(self.sigma))] += 1
                
            
        # Normalize the probability
        sum_states = sum(samples_dict.values())
        normalized_dict = {k: v / sum_states for k, v in samples_dict.items()}
                                                 
        return normalized_dict
    
    def train_metropolis(self, max_epochs = 300, simulation_len = 100, burn_in_len=0, inter_burn_in=0):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0003
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:

            samples_dict = self.metropolis_simulation(simulation_len, burn_in_len, inter_burn_in)
        
            # Calculate the data expectations        
            model_mean, model_correlation = self.get_statistics(samples_dict)
            
            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            oldJ = copy.deepcopy(self.J)
            # Take the step
            self.J = self.J + stepJ
            self.J[self.lind]=0
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            # Calculate the variation of J and H for the termination condition
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%50 == 0 or epoch == max_epochs-1:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
                
            epoch +=1
            
        print('Final stats. TotalParamVariation', round(totalParamVariation, 8))
        print('Total epochs', epoch-1)
        print(model_correlation, 'C Model')
        print(self.data_correlation, 'C Data')

        print(model_mean, 'Average Model')
        print(self.data_mean, 'Average Data')
        print()

        
        print(self.H, 'H')
        print(self.J, 'J')
    
    def create_initial_conditions(self, _num_spins, _num_runs, _simulation_len, _burn_in_len, _inter_burn_in):
        self.num_spins = _num_spins
        
        #Create random H and J matrices
        self.create_H_J()
        
        num_runs = _num_runs
        model_mean_array = np.empty((0,self. num_spins))
        model_correlation_array = np.empty((self.num_spins, self.num_spins, 0))
        for i in range(0,num_runs):
            samples_dict = self.metropolis_simulation(_simulation_len, _burn_in_len, _inter_burn_in)
        
            model_mean, model_correlation = self.get_statistics(samples_dict)
            model_mean_array = np.append(model_mean_array, model_mean.reshape(1, self.num_spins), axis=0)
            model_correlation_array = np.append(model_correlation_array, np.atleast_3d(model_correlation), axis=2)
            
        model_mean = np.mean(model_mean_array, axis=0)
        model_mean_std = np.std(model_mean_array, axis=0)
        model_correlation = np.mean(model_correlation_array, axis=2)
        model_correlation_std = np.std(model_correlation_array, axis=2)

        
        print(self.H, 'iniH')
        print(self.J, 'iniJ')
        
        print(model_mean, 'Mean Data')
        print(model_mean_std, 'Std Data')
        print(model_correlation, 'Corr Data')
        print(model_correlation_std, 'Corr STD')
         
        
        return self.H, self.J, model_mean, model_correlation
        
    

In [8]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]
s_ising = IsingModel(lr=0.1)
s_ising.fit_activations(activations_list)

In [19]:
s_ising.train(10000)

Epoch 100 TotalParamVariation 0.00742333
[[ 0.00000000e+00 -4.75932190e-01  2.50338190e-01 -2.25667336e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50177439e-01 -2.25670117e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.49642655e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50172348e-01 -2.50345797e-01  4.99666174e-01 -7.04193986e-05] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 200 TotalParamVariation 0.00374065
[[ 0.00000000e+00 -4.87670026e-01  2.50070256e-01 -2.37598641e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50044462e-01 -2.37598663e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.93941770e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data


[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 2200 TotalParamVariation 0.00034141
[[ 0.00000000e+00 -4.98862996e-01  2.50000459e-01 -2.48862431e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50000459e-01 -2.48862431e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.81385765e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50000459e-01 -2.50000459e-01  4.99999841e-01  6.50521303e-18] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 2300 TotalParamVariation 0.00032656
[[ 0.00000000e+00 -4.98912423e-01  2.50000419e-01 -2.48911906e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50000419e-01 -2.48911906e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.45300742e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.

Epoch 4200 TotalParamVariation 0.00017875
[[ 0.00000000e+00 -4.99404435e-01  2.50000125e-01 -2.49404281e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50000125e-01 -2.49404281e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.28224557e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50000125e-01 -2.50000125e-01  4.99999957e-01 -5.05780313e-17] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 4300 TotalParamVariation 0.0001746
[[ 0.00000000e+00 -4.99418288e-01  2.50000120e-01 -2.49418141e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50000120e-01 -2.49418141e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.93889390e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data

Epoch 6200 TotalParamVariation 0.00012106
[[ 0.00000000e+00 -4.99596583e-01  2.50000058e-01 -2.49596512e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50000058e-01 -2.49596512e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.42613243e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50000058e-01 -2.50000058e-01  4.99999980e-01 -5.93465164e-17] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 6300 TotalParamVariation 0.00011914
[[ 0.00000000e+00 -4.99602988e-01  2.50000056e-01 -2.49602919e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50000056e-01 -2.49602919e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.01443505e-19]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Dat

In [20]:
initial_ising = IsingModel()
sim_len = 20000
burn_in_len = 100
inter_burn_in = 10
num_runs = 100
num_spins = 4
H, J, model_mean, model_correlation = initial_ising.create_initial_conditions(num_spins, num_runs, sim_len, burn_in_len, inter_burn_in)

[-0.0054536  -0.01546477  0.00982367 -0.01101068] iniH
[[ 0.          0.0043651   0.00096497 -0.01863493]
 [ 0.          0.         -0.00082741 -0.00627001]
 [ 0.          0.          0.          0.00884622]
 [ 0.          0.          0.          0.        ]] iniJ
[-0.00457286 -0.01867337  0.00535678 -0.01033166] Mean Data
[0.01992961 0.02050408 0.02109502 0.02175645] Std Data
[[ 0.          0.00742714 -0.00052261 -0.01745729]
 [ 0.          0.          0.00348744 -0.00850251]
 [ 0.          0.          0.          0.01164824]
 [ 0.          0.          0.          0.        ]] Corr Data
[[0.         0.02082023 0.02261797 0.02213824]
 [0.         0.         0.02233936 0.02475849]
 [0.         0.         0.         0.02215851]
 [0.         0.         0.         0.        ]] Corr STD


In [9]:
s_ising.train_metropolis( max_epochs = 10000, simulation_len = 500, burn_in_len=25,  inter_burn_in=5)

Epoch 50 TotalParamVariation 0.15
[[ 0.         -0.24210526  0.07368421 -0.07368421]
 [ 0.          0.         -0.28421053 -0.38947368]
 [ 0.          0.          0.          0.22105263]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.05263158 -0.22105263  0.30526316  0.07368421] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 100 TotalParamVariation 0.08052632
[[ 0.         -0.55789474  0.32631579 -0.07368421]
 [ 0.          0.         -0.30526316 -0.24210526]
 [ 0.          0.          0.          0.09473684]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.41052632 -0.26315789  0.45263158 -0.11578947] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 150 TotalParamVariation 0.09421053
[[ 0. 

Epoch 950 TotalParamVariation 0.11105263
[[ 0.         -0.6         0.47368421 -0.17894737]
 [ 0.          0.         -0.41052632 -0.22105263]
 [ 0.          0.          0.          0.01052632]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.51578947 -0.45263158  0.53684211  0.01052632] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1000 TotalParamVariation 0.03789474
[[ 0.         -0.41052632  0.28421053 -0.28421053]
 [ 0.          0.         -0.24210526 -0.30526316]
 [ 0.          0.          0.          0.01052632]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.15789474 -0.24210526  0.53684211  0.01052632] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1050 TotalParamVariation 0.088421

Epoch 1850 TotalParamVariation 0.07
[[ 0.         -0.53684211  0.13684211 -0.13684211]
 [ 0.          0.         -0.17894737 -0.32631579]
 [ 0.          0.          0.         -0.07368421]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.32631579 -0.24210526  0.6         0.03157895] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1900 TotalParamVariation 0.09526316
[[ 0.         -0.57894737  0.24210526 -0.41052632]
 [ 0.          0.         -0.32631579 -0.01052632]
 [ 0.          0.          0.         -0.11578947]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.30526316 -0.26315789  0.43157895 -0.13684211] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1950 TotalParamVariation 0.06105263
[[

Epoch 2750 TotalParamVariation 0.07
[[ 0.         -0.49473684  0.32631579 -0.07368421]
 [ 0.          0.         -0.2        -0.43157895]
 [ 0.          0.          0.         -0.03157895]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.28421053 -0.32631579  0.53684211 -0.03157895] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 2800 TotalParamVariation 0.10526316
[[ 0.         -0.64210526  0.28421053 -0.05263158]
 [ 0.          0.         -0.26315789 -0.30526316]
 [ 0.          0.          0.         -0.09473684]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.22105263 -0.03157895  0.43157895 -0.2       ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 2850 TotalParamVariation 0.06052632
[[

Epoch 3650 TotalParamVariation 0.10210526
[[ 0.         -0.38947368  0.09473684 -0.2       ]
 [ 0.          0.         -0.28421053 -0.41052632]
 [ 0.          0.          0.          0.07368421]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.2        -0.26315789  0.22105263  0.09473684] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 3700 TotalParamVariation 0.07631579
[[ 0.         -0.51578947  0.36842105 -0.26315789]
 [ 0.          0.         -0.34736842 -0.22105263]
 [ 0.          0.          0.         -0.09473684]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.28421053 -0.22105263  0.28421053  0.11578947] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 3750 TotalParamVariation 0.11473

...

Epoch 6350 TotalParamVariation 0.07526316
[[ 0.         -0.45263158  0.09473684 -0.17894737]
 [ 0.          0.         -0.22105263 -0.36842105]
 [ 0.          0.          0.         -0.03157895]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.13684211 -0.34736842  0.53684211  0.05263158] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 6400 TotalParamVariation 0.07368421
[[ 0.         -0.43157895  0.11578947 -0.24210526]
 [ 0.          0.         -0.22105263 -0.32631579]
 [ 0.          0.          0.          0.09473684]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.22105263 -0.15789474  0.34736842 -0.05263158] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 6450 TotalParamVariation 0.07684

Epoch 7250 TotalParamVariation 0.09105263
[[ 0.         -0.24210526  0.26315789 -0.41052632]
 [ 0.          0.         -0.13684211 -0.34736842]
 [ 0.          0.          0.         -0.05263158]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.28421053 -0.15789474  0.55789474 -0.03157895] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 7300 TotalParamVariation 0.13421053
[[ 0.         -0.32631579  0.34736842 -0.38947368]
 [ 0.          0.         -0.05263158 -0.28421053]
 [ 0.          0.          0.         -0.2       ]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.28421053 -0.07368421  0.47368421 -0.26315789] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 7350 TotalParamVariation 0.06526

Epoch 8150 TotalParamVariation 0.12315789
[[ 0.         -0.45263158  0.32631579 -0.30526316]
 [ 0.          0.         -0.45263158 -0.24210526]
 [ 0.          0.          0.          0.15789474]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.34736842 -0.47368421  0.68421053  0.17894737] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 8200 TotalParamVariation 0.05157895
[[ 0.         -0.55789474  0.24210526 -0.26315789]
 [ 0.          0.         -0.38947368 -0.17894737]
 [ 0.          0.          0.          0.11578947]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.24210526 -0.26315789  0.57894737 -0.01052632] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 8250 TotalParamVariation 0.08368

Epoch 9050 TotalParamVariation 0.15684211
[[ 0.         -0.49473684  0.49473684 -0.11578947]
 [ 0.          0.         -0.49473684 -0.38947368]
 [ 0.          0.          0.          0.13684211]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.36842105 -0.45263158  0.53684211  0.30526316] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 9100 TotalParamVariation 0.06157895
[[ 0.         -0.62105263  0.36842105 -0.24210526]
 [ 0.          0.         -0.24210526 -0.13684211]
 [ 0.          0.          0.          0.01052632]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.28421053 -0.15789474  0.53684211 -0.07368421] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 9150 TotalParamVariation 0.09052

Epoch 9950 TotalParamVariation 0.13947368
[[ 0.         -0.38947368 -0.03157895 -0.51578947]
 [ 0.          0.         -0.2        -0.09473684]
 [ 0.          0.          0.          0.09473684]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.30526316 -0.2         0.32631579 -0.15789474] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 9999 TotalParamVariation 0.05263158
[[ 0.         -0.49473684  0.22105263 -0.17894737]
 [ 0.          0.         -0.26315789 -0.32631579]
 [ 0.          0.          0.          0.09473684]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.28421053 -0.24210526  0.64210526 -0.05263158] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Final stats. TotalParamVariation 0.052